In [1]:
import re
import sys
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import nltk
from datetime import datetime, timezone
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import demoji

In [2]:
sys.path.append("..")

In [3]:
from src.data.data_cleaning_module import *

In [4]:
nltk.download("rslp");

[nltk_data] Downloading package rslp to /home/gus/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [4]:
columns = ["tweet_text", "sentiment"]

## Leitura de dados brutos (~700k tweets)

In [13]:
df_raw = pd.read_csv("../data/raw/NoThemeTweets.csv", delimiter = ",", usecols = columns)

In [16]:
df_raw.drop_duplicates(["tweet_text"], inplace=True)

### Seleção de 70.000 tweets de forma pseudo aleatória para poupar custo computacional, as proporções de tweets positivos e negativos são mantidas.
#### Essa será a nossa base de dados reduzida.

In [20]:
df_positive = df_raw.loc[df_raw.sentiment=="Positivo", :]
df_negative = df_raw.loc[df_raw.sentiment=="Negativo", :]

In [21]:
df_positive = df_positive.sample(35000, random_state=42);
df_negative = df_negative.sample(35000, random_state=42);

In [32]:
df_raw = pd.concat([df_positive, df_negative], ignore_index=True);

In [33]:
df_raw.sentiment.value_counts()

Positivo    35000
Negativo    35000
Name: sentiment, dtype: int64

### Aplicação de funções para remover links, nomes de usuários dos tweets e caracteres especiais.

In [36]:
df_raw["tweet_text"] = df_raw["tweet_text"].apply(
    lambda tweet: formatar_texto(texto=tweet))

In [37]:
df_raw["tweet_text"] = df_raw["tweet_text"].apply(
    lambda tweet: remove_special_chars(tweet))

### Remoção dos emojis dos textos

In [38]:
df_raw["tweet_text"] = df_raw["tweet_text"].apply(
    lambda tweet: demoji.replace(tweet, "")); # custoso

### Aplicação de função para etiquetar os sentimentos positivos com o valor 1 e negativos com o valor 0

In [39]:
df_raw["sentiment"] = df_raw["sentiment"].replace({"Positivo": 1, "Negativo": 0})

### Adicionando uma coluna com o número de palavras por tweets

In [40]:
df_raw = df_raw.assign(
    number_words=df_raw.tweet_text.apply(lambda x: len(x.split(" "))),
)  # adiciona coluna com número de palavras

### Remoção dos tweets que possuem menos de 5 palavras. Esses tweets curtos podem vir a atrapalhar a etapa de modelagem e portanto serão excluídos

In [41]:
formated_df_raw = df_raw.drop(
    df_raw[df_raw.number_words < 5].index
)  # remove tweets com menos de 5 palavras

### remoção da coluna coluna number_words que não é mais necessária

In [42]:
formated_df_raw.drop(["number_words"], axis=1, inplace=True)

In [43]:
del df_raw # liberando espaço na memória

# Etapa 2 - Geração de 4 DataFrames a partir da remoção ou não das Stopwords, e aplicação de Stemming ou Lematização. 
### DataFrame 1: sem Stopwords e Lematizado; 
### DataFrame 2 : sem Stopwords e com Stemming; 
### DataFrame 3: com Stopwords e Lematizado;
### DataFrame 4: com Stopwords e com Stimming.

<img src="images/Pipeline formatação de texto.png" width=auto heigth=auto>

In [44]:
df_no_stopwords = formated_df_raw.copy()

df_steamed_no_stopwords = formated_df_raw.copy()

df_steamed_no_stopwords["tweet_text"] = df_no_stopwords["tweet_text"].apply(
    lambda tweet: stemming(texto=tweet)
)

df_lemmetized_no_stopwords = formated_df_raw.copy()

df_lemmetized_no_stopwords["tweet_text"] = df_no_stopwords["tweet_text"].apply(
    lambda tweet: lematization(tweet)
)

### Checagem do DataFrame lematizado sem a remoção das stopwords

In [46]:
df_lemmetized_no_stopwords.head(10)

,tweet_text,sentiment
0,capaz q bom q pude ajudar com algo d,1
2,esse serviria para moderar o debate,1
3,o quantidade de cpf cancelar em 2019 ser Imens...,1
5,otimo eu tbm vo de samus fazer um 4x só de samus,1
6,não ter nada para fazer,1
7,só poder ser tu reclamar de si mesmo ser que n...,1
8,ar a contar o dia para eu ir embora e em o últ...,1
9,cf outside nebraska e danny oi esperar que gos...,1
10,sigar maratona de prison Break até de manhã,1
11,oi ufpi estar oficialmente de volta,1


### Checagem do DataFrame com stemming sem a remoção das stopwords

In [47]:
df_steamed_no_stopwords.head(10)

,tweet_text,sentiment
0,capaz q bom q pud ajud com alg d,1
2,ess serv par moder o debat,1
3,a quant de cpf cancel em 2019 ser imens cheg a...,1
5,otim eu tbm vo de samu faz um 4x só de samu,1
6,não tenh nad par faz,1
7,só pod ser tu reclam de si mesm send que nunc ...,1
8,and a cont os dia par me ir emb e no últ dia a...,1
9,cf outsid nebrask e danny oi esper que gost ab...,1
10,sig maraton de prison break até de manhã,1
11,oi ufp est ofic de volt,1


### Leitura das stopwords

In [51]:
stopwords = pd.read_csv("../data/processed/stopwords.csv");

### Remoção das stopwords e aplicação de Lematização e stemming.

In [52]:
df_with_stopwords = formated_df_raw.copy()

df_with_stopwords["tweet_text"] = df_with_stopwords["tweet_text"].apply(
    lambda tweet: remover_stop_words(tweet, stopwords)
)

df_steamed_with_stopwords = formated_df_raw.copy()

df_steamed_with_stopwords["tweet_text"] = df_with_stopwords["tweet_text"].apply(
    lambda tweet: stemming(texto=tweet)
)

df_lemmetized_with_stopwords = formated_df_raw.copy()

df_lemmetized_with_stopwords["tweet_text"] = df_with_stopwords["tweet_text"].apply(
    lambda tweet: lematization(tweet)
)

### Checagem do DataFrame com stemming e com remoção das stopwords

In [54]:
df_steamed_with_stopwords.head(10)

,tweet_text,sentiment
0,capaz q bom q pud ajud alg d,1
2,serv moder debat,1
3,quant cpf cancel 2019 imens cheg saliv p,1
5,otim tbm vo samu faz 4x samu,1
6,nad faz,1
7,pod ser reclam si send nunc problem p,1
8,and cont dia ir emb últ dia ador diz dest está...,1
9,cf outsid nebrask danny oi esper gost abr cap ...,1
10,sig maraton prison break manhã,1
11,oi ufp ofic volt,1


## checagem do dataframe lematizado e com remoção de stopwords

In [53]:
df_lemmetized_with_stopwords.head(10)

,tweet_text,sentiment
0,capaz q bom q pude ajudar algo d,1
2,servir moderar debate,1
3,quantidade cpf cancelar 2019 imensar chego sal...,1
5,otimo tbm vo samus fazer 4x samus,1
6,nada fazer,1
7,poder ser reclamar se ser nunca problema p,1
8,ar contar dias ir embora último dia adorar diz...,1
9,cf outside nebraska danny oi esperar goste abr...,1
10,sigar maratona prison break manhã,1
11,oi ufpi oficialmente voltar,1


### checkagem da integridade do dados processados

In [55]:
print(df_steamed_no_stopwords.shape,df_lemmetized_no_stopwords.shape,df_steamed_with_stopwords.shape,df_lemmetized_with_stopwords.shape)

(57966, 2) (57966, 2) (57966, 2) (57966, 2)


### Salvando os DataFrames na pasta data/processed

In [56]:
df_steamed_no_stopwords.to_csv('../data/processed/df_steamed_no_stopwords.csv', index=False)
df_lemmetized_no_stopwords.to_csv('../data/processed/df_lemmetized_no_stopwords.csv', index=False)
df_steamed_with_stopwords.to_csv('../data/processed/df_steamed_with_stopwords.csv', index=False)
df_lemmetized_with_stopwords.to_csv('../data/processed/df_lemmetized_with_stopwords.csv', index=False)

In [57]:
!ls ../data/processed/

df_lemmetized_no_stopwords.csv	  df_steamed_no_stopwords.csv
df_lemmetized_with_stopwords.csv  df_steamed_with_stopwords.csv
